In [3]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

#

# Code:

In [4]:
# Poplar 1x4". Cut
BlockHeight = 64.0  # [mm]
BlockLength = 68  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


64.0 x 68 x 19.0


In [5]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [6]:
if "Alarm" in cnc.status:
    cnc.home()

In [7]:
cnc.init()

True

# Generate Program


In [8]:
def dot(X=10, Y=10, Z=0, power=1, dwell=1, **kwargs):
    program = GCode.GCode(**kwargs)
    
    program.G0(X=X, Y=Y, Z=Z)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.

    return program

In [10]:
spacings=[1/cnc.x_steps_mm*128, 1/cnc.x_steps_mm*256, 1/cnc.x_steps_mm*512, 1/cnc.x_steps_mm*768, 1/cnc.x_steps_mm*1024]
len(spacings)

5

In [11]:
powers = [25, 50, 75, 100, 150, 200]
len(powers)

6

In [26]:
dwells = [0.1, 0.5]

In [27]:
len(spacings)

5

In [28]:
BlockLength/5

13.6

In [29]:
spacing

0.32

In [32]:
cnc.init()
margin=1 # [mm]
test_len=5 # [mm]
for idx0, power in enumerate(powers):
    X0_next=margin # [mm]
    Y = margin + idx0*5 # [mm]
    assert(Y<BlockHeight)
    for idx1, dwell in enumerate(dwells):
        for idx2, spacing in enumerate(spacings):
            X0=X0_next+(idx2*(test_len)) #[mm]
            Xf=X0+test_len #[mm]
            assert(Xf<BlockLength)
            XX=np.arange(X0, Xf, spacing)
            print("Laserin({}:{}:{}, {}, {}, {})".format(X0, spacing, Xf, Y, power, dwell), end="")
            for X in XX:
                pass
                program = dot(X=X, Y=Y, Z=-14, power=power, dwell=dwell)
                cnc.run(program)
            print("... Done")
        X0_next = Xf+test_len

Laserin(1:0.16:6, 1, 25, 0.1)... Done
Laserin(6:0.32:11, 1, 25, 0.1)... Done
Laserin(11:0.64:16, 1, 25, 0.1)... Done
Laserin(16:0.96:21, 1, 25, 0.1)... Done
Laserin(21:1.28:26, 1, 25, 0.1)... Done
Laserin(31:0.16:36, 1, 25, 0.5)... Done
Laserin(36:0.32:41, 1, 25, 0.5)... Done
Laserin(41:0.64:46, 1, 25, 0.5)... Done
Laserin(46:0.96:51, 1, 25, 0.5)... Done
Laserin(51:1.28:56, 1, 25, 0.5)... Done
Laserin(1:0.16:6, 6, 50, 0.1)... Done
Laserin(6:0.32:11, 6, 50, 0.1)... Done
Laserin(11:0.64:16, 6, 50, 0.1)... Done
Laserin(16:0.96:21, 6, 50, 0.1)... Done
Laserin(21:1.28:26, 6, 50, 0.1)... Done
Laserin(31:0.16:36, 6, 50, 0.5)... Done
Laserin(36:0.32:41, 6, 50, 0.5)... Done
Laserin(41:0.64:46, 6, 50, 0.5)... Done
Laserin(46:0.96:51, 6, 50, 0.5)... Done
Laserin(51:1.28:56, 6, 50, 0.5)... Done
Laserin(1:0.16:6, 11, 75, 0.1)... Done
Laserin(6:0.32:11, 11, 75, 0.1)... Done
Laserin(11:0.64:16, 11, 75, 0.1)... Done
Laserin(16:0.96:21, 11, 75, 0.1)... Done
Laserin(21:1.28:26, 11, 75, 0.1)... Done
Lase

In [34]:
spacings

[0.16, 0.32, 0.64, 0.96, 1.28]

In [35]:
powers

[25, 50, 75, 100, 150, 200]

In [36]:
dwells

[0.1, 0.5]